In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_excel('D:/consumo_caesb.xlsx')
df.head()

,Comp,Sigla,Unidade,Valor,Metro,Observações
0,2021-06-01,UNIRE,UNIRE - UNID DE INTERNAÇÃO DO RECANTO DAS EMAS,289301.98,9684,NaN
1,2021-06-01,UISM,UNIDADE DE INTERNAÇÃO DE SANTA MARIA - UISM,31255.24,2099,NaN
2,2021-06-01,UIP,UIP - UNIDADE DE INTERNAÇÃO PLANALTINA,58892.58,1978,NaN
3,2021-06-01,UIPSS,UIPSS – UNIDADE DE INTERNAÇÃO PROVISÓRIA DE SÃ...,16793.38,570,NaN
4,2021-06-01,SEMI GAMA,UNIDADE DE ATENDIMENTO SEMILIBERDADE GAMA CENTRAL,4026.08,143,NaN


In [3]:
app = dash.Dash(__name__)

In [4]:
app.layout = html.Div([
    html.H1("Dashboard de Consumo de Água"),
    dcc.Dropdown(
        id='tipo-consumo',
        options=[
           # {'label': 'Consumo Total', 'value': 'total'},
            {'label': 'Consumo Médio', 'value': 'medio'},
        ],
        value='medio'
    ),
    dcc.Dropdown(
        id='filtro-unidade',
        options=[{'label': unidade, 'value': unidade} for unidade in df['Sigla'].unique()],
        value=df['Sigla'].unique()[0],  # Valor padrão
        multi=False
    ),
    dcc.Graph(id='grafico-consumo')
])

In [5]:
@app.callback(
    Output('grafico-consumo', 'figure'),
    [Input('tipo-consumo', 'value'),
     Input('filtro-unidade', 'value')]
)
def atualizar_grafico(tipo, unidade):
    # Filtrar o dataframe pela unidade selecionada
    df_filtrado = df[df['Sigla'] == unidade]

    #if tipo == 'total':
    #    fig = px.line(df_filtrado, x='Comp', y='Metro', title=f'Consumo Total de Água - {unidade}')                 
    #else:
    media = df_filtrado['Metro'].median()
    fig = px.bar(df_filtrado, x='Comp', y='Metro', title=f'Consumo Médio de Água - {unidade}', 
                     text=df_filtrado['Metro'].apply(lambda x: f'{x/media:.2%}'))     
               

     # Adicionando a linha da média
    fig.add_hline(y=media, line_dash="dash", line_color="red", 
                      annotation_text="Mediana", annotation_position="bottom right")
        
    return fig

In [6]:
if __name__ == '__main__':
    app.run_server(debug=True)